In [2]:
import torch
import matplotlib.pyplot as plt
import BVP_function as bvp
import numpy as np
import deepxde as dde

import seaborn as sns
sns.set_style("whitegrid") # better looking plots

print("Using GPU:", torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    torch.set_default_tensor_type(torch.cuda.FloatTensor)

from importlib import reload
reload(bvp)

Using GPU: True


<module 'BVP_function' from 'C:\\Users\\u0150568\\PhD\\code\\github\\pinn-cm\\inverse\\BVP_function.py'>

In [4]:
reload(bvp)

lmbd = 1.0
lmbd_trainable = dde.Variable(lmbd - 0.2)
mu = 0.5
mu_trainable = dde.Variable(mu + 0.2)
Q = 4.0

domain = np.array([[0.0, 1.0], [0.0, 1.0]])
geom = dde.geometry.Rectangle([0, 0], [1, 1])

phy_params = {'lmbd': lmbd, 'mu': mu, 'Q': Q}
phy_params_trainable = {'lmbd': lmbd_trainable, 'mu': mu_trainable, 'Q': Q}

#net parameters
net_type = ['Unet','USnet'][1]
n_layers = 4
size_layers = 50
activation = ['tanh','ReLU','Sigmoid'][0]

#loss parameters
loss_type = ['pde','energy'][0] #the energy loss appears to crash LBFGS and provide less accurate results with Adam. PDE loss is therefore used in the following
num_domain = 50**2
train_distribution = ['uniform','pseudo','LHS','Halton','Hammersley','Sobol'][4]

#boundary conditions
bc_type = ['soft','hard'][1]
num_boundary = 50

#optimizer
optimizers = ["adam","L-BFGS"]
learning_rates = [1e-3,None]
iterations = [3000,None]

config = {'net_type':net_type,'n_layers':n_layers,'size_layers':size_layers,'activation':activation,
        'loss_type':loss_type,'num_domain':num_domain,'train_distribution':train_distribution,
        'bc_type':bc_type,'num_boundary':num_boundary,
        'optimizers':optimizers,'learning_rates':learning_rates,'iterations':iterations}  

net_exact = bvp.set_exact_solution(net_type, phy_params,lib='torch')

model, net_wrong_elasticity, pde_net, energy_net, mat_net = bvp.model_setup(geom, config, phy_params_trainable)

In [ ]:
def generate_measures(domain, num_points, func, noise=0.0, seed=None):
    X = np.linspace(domain[0][0], domain[0][1], int(np.sqrt(num_points)))
    Y = np.linspace(domain[1][0], domain[1][1], int(np.sqrt(num_points)))
    Xgrid, Ygrid = np.meshgrid(X, Y)
    X_obs = np.hstack((Xgrid.reshape(-1, 1), Ygrid.reshape(-1, 1)))
    Yexact = func(X_obs)
    Y_obs = Yexact + noise * np.random.randn(*Yexact.shape)
    return X_obs, Y_obs

# generate training data
X_obs, U_obs = generate_measures(domain, 400, U_exact, noise=0.0)

observe_Ux = dde.PointSetBC(X_obs, U_obs[:, 0:1], component=0)
observe_Uy = dde.PointSetBC(X_obs, U_obs[:, 1:2], component=1)

geom = dde.geometry.Rectangle([0, 0], [1, 1])
# material_behavior = dde.icbc.OperatorBC()
bc = [observe_Ux, observe_Uy]

In [5]:
trainable_variables = [lmbd_trainable, mu_trainable]
variable = dde.callbacks.VariableValue(trainable_variables, period=100, filename="elasticity_param.dat")

model.compile("adam", lr=0.001, external_trainable_variables=trainable_variables, metrics=["l2 relative error"])
losshistory, train_state = model.train(iterations=5000,display_every=1000, callbacks=[variable])

model.compile("adam", lr=0.0001, external_trainable_variables=trainable_variables, metrics=["l2 relative error"])
losshistory, train_state = model.train(iterations=5000,display_every=1000, callbacks=[variable])

model.compile("adam", lr=0.0001, external_trainable_variables=trainable_variables, metrics=["l2 relative error"])
losshistory, train_state = model.train(iterations=5000,display_every=1000, callbacks=[variable])

# # train lbfgs
# model.compile("L-BFGS", external_trainable_variables=trainable_variables, metrics=["l2 relative error"])
# losshistory, train_state = model.train(callbacks=[variable])
print(f"lambda:{lmbd_trainable} ; mu: {lmbd_trainable}")

Compiling model...
'compile' took 0.000313 s

Training model...

Step      Train loss                        Test loss                         Test metric   
0         [2.23e+03, 2.67e+02, 2.23e+03]    [2.23e+03, 2.66e+02, 2.29e+03]    [1.14e+00]    
1000      [1.11e+01, 3.13e+00, 6.34e+00]    [1.00e+01, 2.73e+00, 5.66e+00]    [5.91e-01]    
2000      [9.96e-01, 5.10e-01, 1.27e+00]    [8.25e-01, 3.65e-01, 1.09e+00]    [5.91e-01]    
3000      [2.69e-01, 1.48e-01, 4.08e-01]    [1.97e-01, 1.01e-01, 3.32e-01]    [5.99e-01]    
4000      [9.64e-02, 5.19e-02, 1.54e-01]    [6.17e-02, 3.42e-02, 1.20e-01]    [6.06e-01]    
5000      [4.12e-02, 2.44e-02, 7.16e-02]    [2.56e-02, 1.56e-02, 5.41e-02]    [6.13e-01]    

Best model at step 5000:
  train loss: 1.37e-01
  test loss: 9.53e-02
  test metric: [6.13e-01]

'train' took 276.173838 s

Compiling model...
'compile' took 0.000966 s

Training model...

Step      Train loss                        Test loss                         Test metric   
5